# Zadanie – čistá verzia notebooku
Táto verzia je bez výstupov (`_clean.ipynb`) pre prehľadný diff a rýchle nahratie na GitHub.
Použi originálny notebook na plné prepočty a túto čistú verziu pri code review.


# Zadanie 1: Analýza a predspracovanie dát

Zadanie odovzdajte do miesta odovzdania v MS Teams nahraním tohto vypracovaného notebooku a následne potvrdením odovzdania. Bez potvrdenia odovzdania sa zadanie neodošle.

- Dátum a čas zverejnenia: 27.09.2025
- Dátum a čas odovzdania: **26.10.2025 - 23:59**
- Zadanie je hodnotené maximálnym počtom bodov: **20**
- Za každý **načatý deň omeškania** odovzdania zadania **-1 bod**

Úlohou zadania je preukázať a aplikovať vedomosti z oblasti dátovej vedy a to dátovej analýzy a predspracovania dát. Pri vizualizačných technikách využite: histogramy (na určenie distribúcie), box plots (na určenie outlierov), scatter plots (na zobrazenie dát v dvojrozmernej rovine), v prípade potreby použite aj ďalšie vizualizačné techniky.

Zadanie pozostáva z nasledujúcich krokov:
1. Vyberte si jeden z dostupných datasetov:
   - Adult: https://archive.ics.uci.edu/dataset/2/adult
   - Bank Marketing: https://archive.ics.uci.edu/dataset/222/bank+marketing
   - Clickstream Data for Online Shopping: https://archive.ics.uci.edu/dataset/553/clickstream+data+for+online+shopping
   - Heart Disease: https://archive.ics.uci.edu/dataset/45/heart+disease
   - Online Retail: https://archive.ics.uci.edu/dataset/352/online+retail
   - Wine Quality: https://archive.ics.uci.edu/dataset/186/wine+quality
2. Dataset stiahnite a načítajte
3. Podľa dostupného datasetu aplikujte metódy:
   - Deskriptívna analýza
   - Inferenčná analýza (vyberte si jednu techniku)
   - Diagnostická analýza (detailná analýza údajov, korelačná matica)
4. Ak to dataset neurčuje, zvoľte si parameter, na základe ktorého budete deliť dáta do kategórií. Aplikujte metódy vzorkovania a následne zobrazte distribúciu dát podľa zvoleného parametru:
   - vzorkovanie pomocou pravdepodobností (vyberte si jednu metódu)
   - vzorkovanie bez pomoci pravdepodobností (vyberte si jednu metódu)
   - revzorkovanie (vyberte si jednu metódu)
   - podvzorkovanie (vyberte si jednu metódu)
   - prevzorkovanie (vyberte si jednu metódu)
5. Aplikujte metódy manipulácie parametrov:
   - vytvorte nové parametre (aspoň 3)
   - transformujte parametre:
       - vyplňte prázdne hodnoty, ak sú
       - maskujte kategorické parametre (vyberte si jednu metódu)
       - na základe normalizačných techník (vyskúšajte 3 rôzne techniky a porovnajte distribúciu zvoleného parametru)
   - vyskúšajte PCA analýzu:
       - pre nájdenie ideálneho počtu PCA komponentov pomocou dosiahnutia 95% variancie
       - pre vizualizáciu
   - výber parametrov na základe filtrovania: nájdené parametre vypíšte do konzoly 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Načíanie dát z datasetu Wine quality

In [ ]:
df = pd.read_csv('winequality-red.csv', sep=';')

In [ ]:
df.head()

**Deskriptívna analýza**

In [ ]:
print("Pocet riadkov a stlpcov", df.shape)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print("chybajuce hodnoty v datasete:")
print(df.isnull().sum())

print("\nPocet duplicitnych riadkov:", df.duplicated().sum())

In [ ]:
df.hist(figsize=(12,10))
plt.suptitle("Rozdelenie hodnot numerickych premennych")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df)
plt.title("Boxplot – odľahlé hodnoty v dátach vína")
plt.xticks(rotation=45)
plt.show()

Inferenčná analýza

Overenie, či sa obsah alkoholu významne líši medzi vínami s nízkou a vysokou kvalitou.
H₀ (nulová hypotéza): Priemerný obsah alkoholu je rovnaký pre obe skupiny.
H₁ (alternatívna hypotéza): Priemerný obsah alkoholu sa medzi skupinami líši.

In [ ]:
low_quality = df[df['quality'] <= 5]['alcohol']
high_quality = df[df['quality'] >= 6]['alcohol']

print("Počet vín s nízkou kvalitou:", len(low_quality))
print("Počet vín s vysokou kvalitou:", len(high_quality))

In [ ]:
from scipy.stats import ttest_ind

t_stat, p_val = ttest_ind(high_quality, low_quality, equal_var=False)
print("t-statistika:", t_stat)
print("p-hodnota:", p_val)

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='quality', y='alcohol', data=df)
plt.title("Porovnanie alkoholu podľa kvality vína")
plt.xlabel("Kvalita (0–10)")
plt.ylabel("Obsah alkoholu (%)")
plt.show()

Keďže p-hodnota je menšia ako 0.05, odmietame nulovú hypotézu H₀.
To znamená, že existuje štatisticky významný rozdiel v priemernom obsahu alkoholu medzi vínami s nízkou a vysokou kvalitou. Takže vína s vyššou kvalitou mávajú priemerne vyšší obsah alkoholu.

**Diagnostická analýza**

In [ ]:
df.groupby('quality')[['alcohol', 'volatile acidity', 'sulphates', 'citric acid']].mean().round(2)

In [ ]:
corr = df.corr()['quality'].sort_values(ascending=False)
corr

In [ ]:
features = ['alcohol', 'volatile acidity', 'sulphates', 'citric acid']

plt.figure(figsize=(12, 8))
for i, feature in enumerate(features, 1):
    plt.subplot(2, 2, i)
    sns.scatterplot(x=feature, y='quality', data=df, alpha=0.6)
    plt.title(f'Závislosť kvality od {feature}')
plt.tight_layout()
plt.show()

alcohol - stúpa s kvalitou - pozitívny vplyv

volatile acidity - čím viac, tým nižšia kvalita - negatívny vplyv

In [ ]:
from sklearn.linear_model import LinearRegression

X = df[['alcohol', 'volatile acidity', 'sulphates', 'citric acid', 'density']]
y = df['quality']

model = LinearRegression()
model.fit(X, y)

# koeficienty regresie
coefficients = pd.DataFrame({'Premenná': X.columns, 'Koeficient': model.coef_})
coefficients

In [ ]:
selected_features = ['alcohol', 'volatile acidity', 'citric acid', 'sulphates', 'density', 'quality']

sns.pairplot(df[selected_features], hue='quality', palette='coolwarm', diag_kind='kde')
plt.suptitle("Párová analýza vybraných atribútov – Wine Quality", y=1.02)
plt.show()

**Korelačná matica**

In [ ]:
corr_matrix = df.corr()
corr_matrix.round(2)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Korelačná matica – Wine Quality")
plt.show()

In [ ]:
corr_with_quality = corr_matrix['quality'].sort_values(ascending=False)
print(corr_with_quality)

**Vzorkovanie pomocou pravdepodobností
Stratifikovaný výber**

In [ ]:
sns.countplot(x='quality', data=df, palette='coolwarm')
plt.title("Pôvodná distribúcia kvality vín")
plt.xlabel("Kvalita")
plt.ylabel("Počet vzoriek")
plt.show()

In [ ]:
# Stratifikovaný výber – 20 % dát
train, sample_prob = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df['quality'], 
    random_state=42
)

print("Pôvodný počet záznamov:", len(df))
print("Veľkosť vzorky:", len(sample_prob))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,5))
sns.countplot(x='quality', data=df, ax=axes[0], palette='coolwarm')
axes[0].set_title("Pôvodná distribúcia kvality")

sns.countplot(x='quality', data=sample_prob, ax=axes[1], palette='viridis')
axes[1].set_title("Distribúcia kvality po stratifikovanom vzorkovaní")

plt.show()

**Vzorkovanie bez pomoci pravdepodobnosti**

In [ ]:
# Účelový výber: len vína s extrémnou kvalitou (3-4 a 7-8)
vzorka_ucelovy = df[(df['quality'] <= 4) | (df['quality'] >= 7)]

print("Veľkosť pôvodného datasetu:", len(df))
print("Veľkosť účelovej vzorky:", len(vzorka_ucelovy))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

sns.countplot(x='quality', hue='quality', data=df, palette='coolwarm', legend=False, ax=axes[0])
axes[0].set_title("Pôvodná distribúcia kvality")
axes[0].set_xlabel("Kvalita"); axes[0].set_ylabel("Počet")

sns.countplot(x='quality', hue='quality', data=vzorka_ucelovy, palette='viridis', legend=False, ax=axes[1])
axes[1].set_title("Účelová vzorka – extrémne prípady (3–4 a 7–8)")
axes[1].set_xlabel("Kvalita"); axes[1].set_ylabel("Počet")

plt.tight_layout()
plt.show()

In [ ]:
# Porovnanie priemerov vybraných atribútov
vybrane_atributy = ['alcohol', 'volatile acidity', 'sulphates', 'citric acid', 'density']

porovnanie = df.groupby('quality')[vybrane_atributy].mean().round(2)
porovnanie.loc[[3,4,7,8]]

Vína s vyššou kvalitou majú vyšší obsah alkoholu,
nižšiu prchavú kyslosť a odlišné chemické zloženie,
čo zodpovedá záverom predchádzajúcej diagnostickej analýzy.

In [ ]:
sns.boxplot(x='quality', y='alcohol', data=vzorka_ucelovy, palette='coolwarm')
plt.title("Porovnanie obsahu alkoholu v účelovo vybraných kategóriách kvality")
plt.xlabel("Kvalita vína")
plt.ylabel("Obsah alkoholu")
plt.show()

**Revzorkovanie - Bootstrapping**

In [ ]:
mean_alcohol = df['alcohol'].mean()
print("Pôvodný priemer alkoholu:", round(mean_alcohol, 3))

In [ ]:
n = len(df)                 # počet riadkov v datasete
n_iter = 1000               # počet bootstrap opakovaní
bootstrap_means = []        # sem uložíme výsledky

for i in range(n_iter):
    bootstrap_sample = df.sample(n=n, replace=True)  # náhodný výber s návratom
    bootstrap_mean = bootstrap_sample['alcohol'].mean()
    bootstrap_means.append(bootstrap_mean)

bootstrap_means = np.array(bootstrap_means)

In [ ]:
lower = np.percentile(bootstrap_means, 2.5)
upper = np.percentile(bootstrap_means, 97.5)

print("95 % interval spoľahlivosti:", round(lower, 3), "-", round(upper, 3))

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(range(len(bootstrap_means)), bootstrap_means, 
            color='royalblue', alpha=0.6, edgecolor='k', s=30)

# vyznačenie pôvodného priemeru a intervalu
plt.axhline(mean_alcohol, color='red', linestyle='--', linewidth=1.2, label='Pôvodný priemer')
plt.axhline(lower, color='green', linestyle='--', linewidth=1.2, label='95% CI - dolná hranica')
plt.axhline(upper, color='green', linestyle='--', linewidth=1.2, label='95% CI - horná hranica')

plt.title("Bootstrap distribúcia priemeru alkoholu – Scatterplot")
plt.xlabel("Číslo bootstrap vzorky")
plt.ylabel("Priemer alkoholu v danej vzorke")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**Podvzorkovanie**

In [ ]:
df = df.copy()
df['label'] = np.where(df['quality'] >= 6, 'High', 'Low')

print(df['label'].value_counts())

In [ ]:
from imblearn.under_sampling import AllKNN
from sklearn.preprocessing import StandardScaler

X = df.select_dtypes(include=[np.number]).drop(columns=['quality']).values
y = df['label'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

allknn = AllKNN(n_neighbors=3, kind_sel='all', allow_minority=True)
X_resampled, y_resampled = allknn.fit_resample(X_scaled, y)

In [ ]:
import pandas as pd
from collections import Counter

print("Pred podvzorkovaním:", Counter(y))
print("Po podvzorkovaní:", Counter(y_resampled))

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
X_pca_before = pca.fit_transform(X_scaled)
X_pca_after = pca.fit_transform(X_resampled)

# pred podvzorkovaním
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.scatter(X_pca_before[y=='Low', 0], X_pca_before[y=='Low', 1], alpha=0.5, label='Low', color='royalblue')
plt.scatter(X_pca_before[y=='High', 0], X_pca_before[y=='High', 1], alpha=0.5, label='High', color='orange')
plt.title("Pred ALLKNN podvzorkovaním")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.legend()

# po podvzorkovaní
plt.subplot(1,2,2)
plt.scatter(X_pca_after[y_resampled=='Low', 0], X_pca_after[y_resampled=='Low', 1], alpha=0.5, label='Low', color='royalblue')
plt.scatter(X_pca_after[y_resampled=='High', 0], X_pca_after[y_resampled=='High', 1], alpha=0.5, label='High', color='orange')
plt.title("Po ALLKNN podvzorkovaní")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.legend()

plt.tight_layout()
plt.show()

**Prevzorkovanie**

In [ ]:
df = df.copy()
if 'label' not in df.columns:
    df['label'] = np.where(df['quality'] >= 6, 'High', 'Low')

print(df['label'].value_counts())

In [ ]:
X = df.select_dtypes(include=[np.number]).drop(columns=['quality']).values
y = df['label'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=42, k_neighbors=5)

X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

print("Pred prevzorkovaním:", Counter(y))
print("Po prevzorkovaní:", Counter(y_resampled))

In [ ]:
pca = PCA(n_components=2, random_state=42)
X_before = pca.fit_transform(X_scaled)
X_after = pca.fit_transform(X_resampled)

plt.figure(figsize=(12,5))

# pred SMOTE
plt.subplot(1,2,1)
plt.scatter(X_before[y=='Low',0], X_before[y=='Low',1], color='royalblue', alpha=0.5, label='Low')
plt.scatter(X_before[y=='High',0], X_before[y=='High',1], color='orange', alpha=0.5, label='High')
plt.title("Pred SMOTE prevzorkovaním")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.legend()

# po SMOTE
plt.subplot(1,2,2)
plt.scatter(X_after[y_resampled=='Low',0], X_after[y_resampled=='Low',1], color='royalblue', alpha=0.5, label='Low')
plt.scatter(X_after[y_resampled=='High',0], X_after[y_resampled=='High',1], color='orange', alpha=0.5, label='High')
plt.title("Po SMOTE prevzorkovaní (syntetické vzorky)")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.legend()

plt.tight_layout()
plt.show()

**Manipulácia parametrov**

In [ ]:
df = df.copy()

# Pomery medzi dôležitými vlastnosťami vína
df['density_to_alcohol'] = df['density'] / df['alcohol']

# Kombinácia kyslosti
df['acid_index'] = df['fixed acidity'] + df['volatile acidity'] + df['citric acid']

# Pomerný obsah síranov k chloridom
df['sulphate_chloride_ratio'] = df['sulphates'] / df['chlorides']

# kontrola
df[['density_to_alcohol', 'acid_index', 'sulphate_chloride_ratio']].head()

In [ ]:
print("Počet prázdnych hodnôt v datasetu:\n", df.isna().sum())
df = df.fillna(df.mean(numeric_only=True))

**Maskovanie**

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['label_encoded'] = encoder.fit_transform(df['label'])
print(df[['label', 'label_encoded']].drop_duplicates())

**Normalizácia - MinMax, RobustScale, Logaritmická**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler

feature = 'alcohol'
data = df[[feature]].copy()

print("Základné štatistiky pred normalizáciou:")
print(data.describe())

In [ ]:
minmax_scaler = MinMaxScaler()
data['MinMax'] = minmax_scaler.fit_transform(data[[feature]])

robust_scaler = RobustScaler()
data['Robust'] = robust_scaler.fit_transform(data[[feature]])

data['Log'] = np.log1p(data[feature])

data.head()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(14, 4))

sample = data.sample(5, random_state=42).reset_index(drop=True)

# pôvodné dáta
axes[0].plot(sample.index, sample[feature], marker='o', color='blue')
axes[0].set_title("Pôvodné dáta")
axes[0].set_xlabel("Index")
axes[0].set_ylabel("Parameter")

# Min-Max
axes[1].plot(sample.index, sample['MinMax'], marker='o', color='green')
axes[1].set_title("Min-Max škálované dáta")
axes[1].set_xlabel("Index")
axes[1].set_ylabel("Škálované hodnoty [0, 1]")

# Robust
axes[2].plot(sample.index, sample['Robust'], marker='o', color='darkorange')
axes[2].set_title("Robust škálované dáta")
axes[2].set_xlabel("Index")
axes[2].set_ylabel("Normalizované hodnoty")

# Logaritmická
axes[3].plot(sample.index, sample['Log'], marker='o', color='purple')
axes[3].set_title("Log-transformované dáta")
axes[3].set_xlabel("Index")
axes[3].set_ylabel("log(1+x)")

plt.tight_layout()
plt.show()


**PCA analýza**

In [ ]:
from sklearn.decomposition import PCA
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in ['quality', 'label_encoded']:
    if col in num_cols:
        num_cols.remove(col)

X = df[num_cols].values

# štandardizácia – PCA vyžaduje rovnakú mierku pre všetky atribúty
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Počet použitých atribútov:", len(num_cols))

In [ ]:
pca = PCA().fit(X_scaled)

explained_var = np.cumsum(pca.explained_variance_ratio_)

# zistenie počtu komponentov pre 95 % variance
n_components_95 = np.argmax(explained_var >= 0.95) + 1
print(f"Ideálny počet PCA komponentov pre 95 % vysvetlenej variance: {n_components_95}")

# graf
plt.figure(figsize=(7,5))
plt.plot(range(1, len(explained_var)+1), explained_var, marker='o', color='royalblue')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% hranica')
plt.axvline(x=n_components_95, color='g', linestyle='--', label=f'{n_components_95} komponentov')
plt.title("Kumulatívna vysvetlená variancia PCA")
plt.xlabel("Počet komponentov")
plt.ylabel("Podiel vysvetlenej variance")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# aplikácia PCA s 2 komponentmi pre vizualizáciu
pca_2d = PCA(n_components=2)
X_pca = pca_2d.fit_transform(X_scaled)

plt.figure(figsize=(7,6))
plt.scatter(X_pca[:,0], X_pca[:,1],
            c=df['label_encoded'], cmap='coolwarm', alpha=0.7)
plt.title("PCA vizualizácia – prvé dve komponenty")
plt.xlabel("Hlavná komponenta 1")
plt.ylabel("Hlavná komponenta 2")
plt.colorbar(label="label (0=Low, 1=High)")
plt.tight_layout()
plt.show()


In [ ]:
corr = df[num_cols + ['quality']].corr(numeric_only=True)['quality'].sort_values(ascending=False)
print("Parametre najviac korelujúce s kvalitou:\n")
print(corr.head(10))